这一小节我们演示如何使用千帆 SDK 在调用大模型服务时添加历史对话信息，如何使用 LangChain 提供的 Memory 组件来管理历史信息的存储方式。

In [1]:
import os
import qianfan

os.environ['QIANFAN_AK'] = "your_ak"
os.environ['QIANFAN_SK'] = "your_sk"

下面演示通过 qianfan.Messages() 对象传递给大模型历史对话信息

In [2]:
chat_comp = qianfan.ChatCompletion()

# 下面是一个与用户对话的例子
msgs = qianfan.Messages()
msgs.append("你好，我的名字是千帆职能小助手") # 增加用户输入
resp = chat_comp.do(messages=msgs)
print(resp["body"]["result"]) # 模型的输出
msgs.append(resp)             # 追加模型输出

您好，千帆职能小助手！请问有什么我可以帮助您的吗？


In [3]:
msgs.append("中国的首都是哪里？") # 增加用户输入
resp = chat_comp.do(messages=msgs)
print(resp["body"]["result"]) # 模型的输出
msgs.append(resp)             # 追加模型输出

中国的首都是北京。


In [4]:
msgs.append("我叫什么名字？") # 增加用户输入
resp = chat_comp.do(messages=msgs)
print(resp["body"]["result"]) # 模型的输出
msgs.append(resp)  

您告诉我，您叫千帆职能小助手。


下面演示通过 LangChain 的 Memory 组件来管理历史信息

In [2]:
from langchain.chat_models import QianfanChatEndpoint
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [3]:
llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.01)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)

In [4]:
conversation.predict(input = "你好，我的名字是千帆职能小助手")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好，我的名字是千帆职能小助手
AI:

> Finished chain.


'你好，千帆职能小助手！很高兴认识你。我有什么可以帮助您的吗？'

In [5]:
conversation.predict(input = "中国的首都是哪里？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我的名字是千帆职能小助手
AI: 你好，千帆职能小助手！很高兴认识你。我有什么可以帮助您的吗？
Human: 中国的首都是哪里？
AI:

> Finished chain.


'中国的首都是北京。它是一个历史悠久且文化丰富的城市，位于中国北部。作为中国的政治、文化和经济中心，北京吸引着成千上万的游客前来参观和体验其独特的魅力。'

In [6]:
conversation.predict(input = "我的名字是什么？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我的名字是千帆职能小助手
AI: 你好，千帆职能小助手！很高兴认识你。我有什么可以帮助您的吗？
Human: 中国的首都是哪里？
AI: 中国的首都是北京。它是一个历史悠久且文化丰富的城市，位于中国北部。作为中国的政治、文化和经济中心，北京吸引着成千上万的游客前来参观和体验其独特的魅力。
Human: 我的名字是什么？
AI:

> Finished chain.


'您的名字是千帆职能小助手。很高兴再次和您交流！有什么我可以帮助您的吗？'

In [7]:
print(memory.buffer)

Human: 你好，我的名字是千帆职能小助手
AI: 你好，千帆职能小助手！很高兴认识你。我有什么可以帮助您的吗？
Human: 中国的首都是哪里？
AI: 中国的首都是北京。它是一个历史悠久且文化丰富的城市，位于中国北部。作为中国的政治、文化和经济中心，北京吸引着成千上万的游客前来参观和体验其独特的魅力。
Human: 我的名字是什么？
AI: 您的名字是千帆职能小助手。很高兴再次和您交流！有什么我可以帮助您的吗？


In [8]:
memory.save_context({"input": "你好！"}, {"output": "你好呀，近来身体如何？"})

In [9]:
print(memory.buffer)

Human: 你好，我的名字是千帆职能小助手
AI: 你好，千帆职能小助手！很高兴认识你。我有什么可以帮助您的吗？
Human: 中国的首都是哪里？
AI: 中国的首都是北京。它是一个历史悠久且文化丰富的城市，位于中国北部。作为中国的政治、文化和经济中心，北京吸引着成千上万的游客前来参观和体验其独特的魅力。
Human: 我的名字是什么？
AI: 您的名字是千帆职能小助手。很高兴再次和您交流！有什么我可以帮助您的吗？
Human: 你好！
AI: 你好呀，近来身体如何？


通过附带窗口的记忆组件，还可以设置大模型记忆的最大对话轮数，防止上下文过长而导致的内存超限问题。

In [10]:
from langchain.memory import ConversationBufferWindowMemory

通过设置窗口大小为1可以保证始终只存储最近一轮的对话记录

In [14]:
memory = ConversationBufferWindowMemory(k=1)

In [15]:
memory.save_context({"input": "你好！"}, {"output": "你好呀，近来身体如何？"})
memory.save_context({"input": "我的身体很好，你怎么样"}, {"output": "我也很好"})

In [16]:
print(memory.buffer)

Human: 我的身体很好，你怎么样
AI: 我也很好


In [17]:
conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)
conversation.predict(input = "你好，我的名字是千帆职能小助手")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 我的身体很好，你怎么样
AI: 我也很好
Human: 你好，我的名字是千帆职能小助手
AI:

> Finished chain.


'你好，千帆职能小助手。很高兴认识你。有什么我可以帮助你的吗？'

In [20]:
conversation.predict(input = "中国的首都是哪里？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我的名字是千帆职能小助手
AI: 你好，千帆职能小助手。很高兴认识你。有什么我可以帮助你的吗？
Human: 中国的首都是哪里？
AI:

> Finished chain.


'中国的首都是北京。它是一个历史悠久且文化丰富的城市，位于中国北部。北京作为中国的政治、文化和经济中心已有数百年的历史。'

In [21]:
conversation.predict(input = "我的名字是什么？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 中国的首都是哪里？
AI: 中国的首都是北京。它是一个历史悠久且文化丰富的城市，位于中国北部。北京作为中国的政治、文化和经济中心已有数百年的历史。
Human: 我的名字是什么？
AI:

> Finished chain.


'对不起，我不知道您的名字。您没有提供给我这个信息。如果您告诉我您的名字，我会很高兴地与您交流。'

通过使用附带摘要功能的记忆组件，可以让大模型始终关注上下文的关键信息而不是其中的细节。

In [51]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm)

In [47]:
memory.save_context({"input": "你好！"}, {"output": "你好呀，近来身体如何？"})
memory.save_context({"input": "我的身体很好，你怎么样"}, {"output": "我也很好"})
memory.save_context({"input": "最近有什么新闻"}, {"output": "2024年春节至少放假8天，从初一放到初八，如果配合带薪休假可以放假9天，堪称新年第一份大礼包。虽然除夕当天名义上不放假，但鼓励各单位和机构根据实际情况来灵活安排。同时高速公路已经免费了，实质上就是创造条件给老百姓多放一天假。春节假期的延长，能够满足广大群众有更多时间用于亲人团聚的心愿，还可以实现一个假期先团聚、后旅游的小目标。这是在依据《全国年节及纪念日放假办法》规定的基础上的创新之举、暖心之举，充分体现出政策制定者的用心良苦和精心设计。"})

In [48]:
print(memory.buffer)

The human asks the AI about recent news. The AI responds by discussing the extension of the Spring Festival holiday in 2024 to at least 8 days, with the potential for 9 days if combined with paid leave. This is intended to allow for more time for family reunions and travel. The AI mentions that the highway is now free, effectively adding another day off for people. The extension is seen as a positive and thoughtful move by policy makers.


这里记忆模块对之前对话的总结是英文的，原因是摘要默认的 Prompt 模版是英文的，我们可以将 Prompt 模版打印出来。

In [52]:
print(memory.prompt)

input_variables=['new_lines', 'summary'] template='Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.\n\nEXAMPLE\nCurrent summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.\n\nNew lines of conversation:\nHuman: Why do you think artificial intelligence is a force for good?\nAI: Because artificial intelligence will help humans reach their full potential.\n\nNew summary:\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\nEND OF EXAMPLE\n\nCurrent summary:\n{summary}\n\nNew lines of conversation:\n{new_lines}\n\nNew summary:'


通过配置中文自定义 Prompt 模版可以更加适配在中文语料上训练的大模型

In [56]:
from langchain.prompts.prompt import PromptTemplate

SUMMARIZER_TEMPLATE = """逐步总结所提供的谈话内容，将其添加到上一个摘要中，返回一个新的摘要。

示例
当前摘要：
人类会问人工智能对人工智能的看法。人工智能认为人工智能是一股向善的力量。

新的谈话内容：
人类：为什么你认为人工智能是一股向善的力量？
人工智能：因为人工智能将帮助人类充分发挥潜力。

新摘要：
人类会问人工智能对人工智能的看法。人工智能认为人工智能是一股向善的力量，因为它将帮助人类充分发挥潜力。

示例结束
当前摘要：
{summary}

新的谈话内容：
{new_lines}

新摘要：
"""
SUMMARY_PROMPT = PromptTemplate(
    input_variables=["summary", "new_lines"], template=SUMMARIZER_TEMPLATE
)
memory = ConversationSummaryMemory(llm=llm, prompt=SUMMARY_PROMPT)

memory.save_context({"input": "你好！"}, {"output": "你好呀，近来身体如何？"})
memory.save_context({"input": "我的身体很好，你怎么样"}, {"output": "我也很好"})
memory.save_context({"input": "最近有什么新闻"}, {"output": "2024年春节至少放假8天，从初一放到初八，如果配合带薪休假可以放假9天，堪称新年第一份大礼包。虽然除夕当天名义上不放假，但鼓励各单位和机构根据实际情况来灵活安排。同时高速公路已经免费了，实质上就是创造条件给老百姓多放一天假。春节假期的延长，能够满足广大群众有更多时间用于亲人团聚的心愿，还可以实现一个假期先团聚、后旅游的小目标。这是在依据《全国年节及纪念日放假办法》规定的基础上的创新之举、暖心之举，充分体现出政策制定者的用心良苦和精心设计。"})

print(memory.buffer)

人类和人工智能互相问候，询问身体状况。双方都表示身体状况良好。AI分享了2024年春节放假至少8天的新闻，鼓励各单位灵活安排，可实现先团聚后旅游的目标，展现出政策制定者的用心良苦和精心设计。


In [57]:
conversation = ConversationChain(
    llm = llm,
    memory = memory,
    verbose = True
)
conversation.predict(input="2024年春节放假几天？")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
人类和人工智能互相问候，询问身体状况。双方都表示身体状况良好。AI分享了2024年春节放假至少8天的新闻，鼓励各单位灵活安排，可实现先团聚后旅游的目标，展现出政策制定者的用心良苦和精心设计。
Human: 2024年春节放假几天？
AI:

> Finished chain.


'**至少8天**。具体的安排是**1月21日至27日**放假调休，**1月28日（星期六）、1月29日（星期日）上班**。这样的设计可以让人们有充足的时间与家人团聚，然后再出去旅游。这样的安排展现了政策制定者的用心良苦和精心设计，他们不仅考虑到了人们的家庭需求，也考虑到了旅游市场的需求。'